
# **Start here!**

## A list of useful resources
* [CRyPTIC data tables](https://ftp.ebi.ac.uk/pub/databases/cryptic/release_june2022/reproducibility/data_tables/cryptic-analysis-group/). See README and data-model (DATA_SCHEMA.pdf).
* [Multidrug-resistant tuberculosis
](https://www.nature.com/articles/s41572-024-00504-2). Published in Nature Reviews Disease Primers.
* [Global Tuberculosis Report 2023](https://iris.who.int/bitstream/handle/10665/373828/9789240083851-eng.pdf?sequence=1). By the WHO.
* [Catalogue of mutations in M. tb](https://www.who.int/publications/i/item/9789240028173). Also by the WHO.
* [GWAS of M. tb](https://pubmed.ncbi.nlm.nih.gov/35944070/). By CRyPTIC, published 2022.
* [H37Rv Genome assembly](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000195955.2/). From NCBI Datasets. Also divides the genome by each gene. [Gene annotations](https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_000195955.2/) are also available. Note the "Sequence and Annotation" .gbff format, which will be used later.

Note: This notebook is a record of a lot of work! But it's become quite unwieldy. So we will now create a notebook per experiment.

## Utilities

In [1]:
"""
Mount Drive onto this notebook. Note that there is a specific
file structure. In particular, we have
-- MyDrive
-- -- EVO
-- -- -- sample_vcfs
-- -- -- CRyPTIC_reuse_table_20231208.csv
-- -- -- h37rv_genebank_flatfile.gbff

See comments to obtain any missing files.
"""

import pickle
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

evo_general_dir = '/content/drive/MyDrive/EVO/'

samples_dir = 'vcfs/'

# obtain via wget: ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/195/955/GCA_000195955.2_ASM19595v2/
h37rv_genome_file = 'GCF_000195955.2_ASM19595v2_genomic.fna'

# obtain via wget: ftp.ebi.ac.uk/pub/databases/cryptic/release_june2022/reuse/
cryptic_general_file = 'CRyPTIC_reuse_table_20231208.csv'

# note: you can use Python variables in terminal, e.g. in the next section we use
# wget -P $output_dir $vcf_file
# where output_dir and vcf_file are variables

Mounted at /content/drive


In [2]:
import numpy as np

for k in range(4):
  a = '/content/drive/MyDrive/EVO/emb_embeddings_v1/left_lr_confusion_' + str(k) + '.npy'
  m = print(np.load(a))
  print(m)

[[ 301    0    0]
 [ 468    0    0]
 [1651    0    0]]
None
[[ 301    0    0]
 [ 468    0    0]
 [1651    0    0]]
None
[[ 301    0    0]
 [ 468    0    0]
 [1651    0    0]]
None
[[ 301    0    0]
 [ 468    0    0]
 [1651    0    0]]
None


In [ ]:
"""
Open/close zipped VCF files. If a file if unzipped, be sure
to re-zip it after. (If you forget, then gunzip/gzip will complain
about not finding a file, which isn't disasterous.)

Note: pd.read_csv can open zipped files by passing in compression="gzip".
"""

# to use gunzip / gzip, your computer must use the correct encoding (UTF-8)
# unclear why, but colab does not default to this encoding
import locale

# # check what encoding is currently in use
# print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

def gunzip(file_path):
  !gunzip -f $file_path

def gzip(file_path):
  !gzip -f $file_path

def extract_site(unique_id):
  return unique_id.split(".")[1]

# def get_sample_file_path(row):
#   sample_id = row['UNIQUEID']
#   site_dir = "site_" + str(extract_site(sample_id)) + "/"
#   return evo_general_dir + samples_dir + site_dir + row['VCF'].split('/')[-1]

# def open_vcf(file_path):
#   """
#     This is useful if you would like a Pandas DF. You can also use
#     gumpy to open/manage VCF files.
#   """
#   f = open(file_path)

#   # VCF files have a detailed header; each row of the header begins with ##
#   # The final row is the column header; it begins with #
#   cols = []
#   while '#' in f.readline():
#     cols = f.readline().strip().split('\t')
#     cols[0] = cols[0][1:]
#   df_vcf = pd.read_csv(f, delimiter='\t', header=0, names=cols)

#   return df_vcf

# # how to unzip or zip a file given a row in our df
# file_path = get_sample_file_path(row)
# gunzip(file_path)

# # note the gunzipped file will no longer have the .gz ending
# # therefore, to use the gunzipped file, we use file_path[:-3]
# gzip(file_path[:-3])

## Download data

* Download 'CRyPTIC_reuse_table_20221019.csv' to obtain FTP URLs
* Set output_dir to wherever your heart desires

There are 12,289 VCF files, so downloading make take awhile. Each compressed file is between 25 to 50KB in size. (That's about 500 MB.) Hopefully, you only need to run this section once.

Note: A more robust data download notebook can be found [here](https://colab.research.google.com/drive/1gFBrTnvoNG5NItlxxe85dEd_dKXh_k8_?usp=sharing). The code here may already have been deprecated.






In [ ]:
import pandas as pd
df = pd.read_csv(evo_general_dir + cryptic_general_file)

ftp_dir = 'ftp.ebi.ac.uk/pub/databases/cryptic/release_june2022/reproducibility/'
output_dir = evo_general_dir + 'sample_vcfs'

In [ ]:
# """
#   Uncomment the below to download CRyPTIC genotyping data (VCFs).
# """

# import os
# import pandas as pd

# from tqdm import tqdm


# def extract_site(vcf_path):
#   vcf_path = vcf_path[vcf_path.find("site"):]
#   return vcf_path.split(".")[1]

# for index, row in df[['VCF']].iterrows():
#   vcf_path = ftp_dir + row[0]
#   output_dir = evo_general_dir + 'sample_vcfs' + '/site_' + extract_site(vcf_path)

#   if not os.path.exists(output_dir):
#     !mkdir $output_dir

#   !wget -P $output_dir -q $vcf_path

# Experiment 1: Determine input sequence length

~~We can divide (the vast majority of) our Tb genomes into 4 lineages. (Some samples may be in between lineages). Do Tb in the same lineage have similar embeddings?~~

Our original intent was to see how embeddings reflect lineage. However, this became a non-trivial task because of compute restraints. Thus, this experiment turned into one about how we should actually embed sequences.

**Result:** Failure! Given a 1000-bp sequence $a$ and an identical sequence $b$, adding a single SNP to $b$ will not change its encoding.

**Gameplan:** Instead of looking at large segments of the genome, confine ourselves to small segments.

Note: The code below will not work anymore, since some tools (e.g. GenomeLoader) have been deprecated (and replaced with superior tools).


In [ ]:
# # download lineage file from CRyPTIC GWAS study: https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3001755
# ftp_dir = 'ftp.ebi.ac.uk/pub/databases/cryptic/release_june2022/pubs/gwas2022/data/cryptic_R1_10422_protein_kmer_kinship_to_distance_matrix_NJ_tree_information.txt'
# output_dir = evo_general_dir

# !wget -P $output_dir $ftp_dir

In [ ]:
df_lineage = pd.read_csv('/content/drive/MyDrive/EVO/cryptic_R1_10422_protein_kmer_kinship_to_distance_matrix_NJ_tree_information.txt', delimiter='\t')
df_rif = pd.read_csv(evo_general_dir + 'rif_resistant_genes.csv')

In [ ]:
def get_fragment(df_vcf, index, genome_kvs, bp_down, bp_up):
  """
    For testing....

    In the future, this can be repurposed to nab specific areas
    of a genome.

    inputs:
    - df_vcf: DataFrame representing the body of a VCF file
    - index: row of VCF file
    - genome_kvs: dict that stores genome fragments created by GenomeLoader
    - bp_down, bp_up: positive integers (i.e. > 0)
  """
  key_after = df_vcf.loc[index]['POS'] + len(df_vcf.loc[index]['REF']) - 1
  key_before = df_vcf.loc[index - 1]['POS'] + len(df_vcf.loc[index - 1]['REF']) - 1

  frag_before = genome_kvs[key_before]
  frag_after = genome_kvs[key_after]

  return frag_before[-bp_down:] + frag_after[:bp_up]

# Example tests, need to build out this section
# pos = df_vcf.loc[525]['POS']
# print(gl.reference_genome[pos+7:pos+28])

# print(get_fragment(df_vcf, 525, gl.frag_genomes[cols[-1]], 6, 20)[:])

class GenomeLoader():
  def __init__(self, whole_genome_path=None, reference_genome=None):
    self.whole_genome_path = whole_genome_path
    self.reference_genome = reference_genome
    self.frag_genomes = dict()

    if self.whole_genome_path:
      with open(whole_genome_path) as f:
        metadata = f.readline().strip()
        lines = [line.strip() for line in f]
        self.reference_genome = "".join(lines)

    assert self.reference_genome, "GenomeLoader did not receive a reference genome"


  def vcf_to_genome(self, vcf_file):
    """
      Given a vcf_file, return a whole genome.

      Here is the general strategy:
      Think of the reference genome as a series of fragments;
      we create fragments such that each fragment ends in a mutation
      according to the vcf_file.

      Then, to re-construct a whole genome, we append these fragments
      together. Done!
    """
    f = open(vcf_file)

    # VCF files have a detailed header; each row of the header begins with ##
    # The final row is the column header; it begins with #
    cols = []
    while '#' in f.readline():
      cols = f.readline().strip().split('\t')
    cols[0] = cols[0][1:]

    df_vcf = pd.read_csv(f, delimiter='\t', header=0, names=cols)

    genome_kvs = dict()
    current_pos = 0
    for index, row in df_vcf.iterrows():
      current_fragment = self.reference_genome[current_pos:row['POS'] - 1]
      current_fragment += row['ALT']

      genome_kvs[current_pos] = current_fragment
      current_pos = row['POS'] + len(row['REF']) - 1

    genome = ""
    for key, value in genome_kvs.items(): #requires Python 3.7
      genome += value

    self.frag_genomes[cols[-1]] = genome_kvs

    return genome

In [ ]:
import pandas as pd
df = pd.read_csv(evo_general_dir + cryptic_general_file)
gl = GenomeLoader(evo_general_dir + h37rv_genome_file)

df_lineage_one = df_lineage[df_lineage.Lineage == "Lineage 1"]
df_lineage_three = df_lineage[df_lineage.Lineage == "Lineage 3"]

In [ ]:
def open_vcf(file_path):
  f = open(file_path)

  # VCF files have a detailed header; each row of the header begins with ##
  # The final row is the column header; it begins with #
  cols = []
  while '#' in f.readline():
    cols = f.readline().strip().split('\t')
    cols[0] = cols[0][1:]
  df_vcf = pd.read_csv(f, delimiter='\t', header=0, names=cols)

  return df_vcf

In [ ]:
genomes = []
vcfs = []
for index, row in tqdm(df_lineage_three.iterrows()):
  row = df[df.UNIQUEID == row['uniqueID']].iloc[0, :]
  file_path = get_sample_file_path(row)
  gunzip(file_path)

  genomes.append(gl.vcf_to_genome(file_path[:-3]))
  vcfs.append(open_vcf(file_path[:-3]))

  gzip(file_path[:-3])
  if index > 1000: break

In [ ]:
cysA3_seqs = []
cysA3_start = 3483974
cysA3_end = 3484807

unique_pos = []

for i, vcf in enumerate(vcfs):
  temp = vcf.query('POS <= 3483974')

  diff = 0
  for index, row in temp.iterrows():
    diff += len(row['ALT']) - len(row['REF'])

  unique_pos.append(row['POS'])
  cysA3_seqs.append(genomes[i][cysA3_start - 1 + diff: cysA3_end + diff])

In [ ]:
!pip install evo-model

In [ ]:
from evo import Evo
import torch

device = 'cuda:0'

# evo_model = Evo('evo-1-131k-base') # load model
evo_model = Evo('evo-1-8k-base') # load model
model, tokenizer = evo_model.model, evo_model.tokenizer
model.to(device)
model.eval()

# Note to self: How does this code work?
# How to extract embeddings: https://github.com/evo-design/evo/issues/32
from torch import nn

class CustomEmbedding(nn.Module):
  def unembed(self, u):
    return u

model.unembed = CustomEmbedding()

# Basically, this removes the last part of Evo
# so that the embeddings are not turned into logits.

In [ ]:
!pip install gputil

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

In [ ]:
def run(model, tokenizer, sequence):
  input_ids = torch.tensor(
      tokenizer.tokenize(sequence),
      dtype=torch.int,
  ).to(device).unsqueeze(0)

  embed, _ = model(input_ids) # (batch, length, embed dim)

  print('Embed: ', embed)
  print('Shape (batch, length, embed dim): ', embed.shape)

  return embed.to(dtype=torch.float64).detach().cpu().numpy()

In [ ]:
cysA3_seqs = set(cysA3_seqs)
len(cysA3_seqs)

In [ ]:
# run(model, tokenizer, sequence = genomes[6][:1450])
embeddings_three = []
cysA3_seqs = set(cysA3_seqs)
for seq in tqdm(cysA3_seqs):
  if len(seq) > 1000: seq = seq[:1000]
  seq = seq[:5]

  embeddings_three.append(run(model, tokenizer, seq))

# Experiment 2: Predict RIF resistance with an embedding of RR-DR

We need to identify a way to create meaningful embeddings. Our first attempt will be to look at loci with common mutations.

For now, we focus on Rifampicin (RIF) resistance. Per the WHO, only mutations in *rpoB* are known to confer RIF resistance. Thus, we focus on creating meaningful *rpoB* embeddings.

**Result:** Success! We can predict RIF resistance with sensitivity 0.96% and specificity 0.89%.

## Gene Embedding Factory

This is a wrapper class that organizes the use of Evo and Gumpy.

In [ ]:
!pip install evo-model

import torch
from evo import Evo

!pip install gumpy

import gumpy as gp
from collections import Counter


class CustomEmbedding(torch.nn.Module):
  """
    Monkey patch to obtain Evo model embeddings,
    instead of logits.
  """
  def unembed(self, u):
    return u


class GeneEmbeddingFactory():
  """
    Just a wrapper around Evo. Because we call Evo often enough, this
    makes our lives marginally better.

    22.5 GB of GPU RAM is barely enough to perform inference on 500-length
    sequences. (Storing the model requires ~12, running inference requires ~10.)
  """
  def __init__(self,
               ref_genome, # Genome
               version='evo-1-8k-base',
               device='cuda:0',
               logits=False):

    self.ref_genome = ref_genome

    evo_model = Evo(version)
    self.model = evo_model.model
    self.tokenizer = evo_model.tokenizer

    self.device = device
    self.model.to(device)
    self.model.eval()

    if not logits:
      self.model.unembed = CustomEmbedding()


  def get_gene(self, vcf, gene_name):
    """
      Note: Unless you have a lot of RAM, it is not possible
      load 12,228 whole M. tb genomes. Hence, we work on a gene-by-gene
      basis.

      input:
      - vcf: gumpy VCFFile object, M. tb VCF
      - gene_name: string

      output:
      - gene: gumpy Gene object
    """
    genome = self.ref_genome + vcf

    return genome.build_gene(gene_name)

  def run(self,
          sequence,
          to_cpu=True):
    input_ids = torch.tensor(
        self.tokenizer.tokenize(sequence),
        dtype=torch.int,
    ).to(self.device).unsqueeze(0)

    embed, _ = self.model(input_ids) # (batch, length, embed dim)
    if to_cpu: embed = embed.to(dtype=torch.float64).detach().cpu().numpy()

    return embed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu

## Quick and dirty analysis of *rpoB* variants

We use "primers" to find where we should extract data a whole genome sequence. For example, to study rpoB, we find the rifampicin resistance determining region by searching for "CGATCACACCGCAGACGTTGA" and "CAGACGTTGATCAACATCCG" in a sample's WGS. (These are two established primers used to study M. tb DNA.)

In [ ]:
import pickle
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

evo_general_dir = '/content/drive/MyDrive/EVO/'
vcfs_dir = 'vcfs/'
cryptic_dir = 'cryptic_data/'
cryptic_reuse_csv = 'CRyPTIC_reuse_table_20231208.csv'

input_dir = evo_general_dir + vcfs_dir
reuse_vcf = pd.read_csv(evo_general_dir + cryptic_reuse_csv)

# for simplicity, keep only the columns we are interested in
reuse_vcf = reuse_vcf[['UNIQUEID', 'VCF', 'RIF_BINARY_PHENOTYPE', 'RIF_PHENOTYPE_QUALITY', 'RIF_MIC']]

bp_up = 250
bp_down = 250

# Did you know? There is an 81-bp section of the M. tb genome
# known as the riframin resistance determining region (RRDR).
# We will be embedding the RRDR for our first experiment.
tb_rrdr_primer1 = 'CGATCACACCGCAGACGTTGA'
tb_rrdr_primer2 = 'CAGACGTTGATCAACATCCG'

# the RRDR is in the rpoB gene
gene_name = 'rpoB'

nucleotide_map = {
    'A': 'T',
    'T': 'A',
    'C': 'G',
    'G': 'C'
}

In [ ]:
"""
  Note: Often, the parts related to Evo are commented out in GeneEmbeddingFactory.
  This is because loading the Evo model is a pain, and requires a nice GPU (L4 or A100 in Colab).
  If embedding production is required, then be sure to uncomment.
"""
# ref_genome = gp.Genome(evo_general_dir + 'h37rv_genebank.gbk', is_reference=True)
# pickle.dump(ref_genome, open(evo_general_dir + 'h37rv_genebank.pkl', 'wb'))
ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

In [ ]:
"""
  Creates a list of 500-bp sequences using primers.

  Note: Originally, data was stored in increments of 1000. However,
  because of the size of the embeddings (500, 4096), this would
  result in files as large as 15 GB. So, additional code was written
  to create files in increments of 100.
"""

# def get_site(unique_id):
#   return unique_id.split('.')[1]

# def get_start(seq, primers):
#   for i, primer in enumerate(primers):
#     if primer in seq:
#       return seq.find(primer), i
#   return -1, -1

# primers = [tb_rrdr_primer1, tb_rrdr_primer2]
# primers_used = [[] for _ in range(len(primers))]

# unique_ids = []
# seqs = []
# genome_dir = 'genomes/'
# for i, row in tqdm(reuse_vcf.iterrows()):
#   if row['UNIQUEID'] in unique_ids: continue

#   sample_directory = evo_general_dir + genome_dir + "site_" + get_site(row['UNIQUEID']) + '/'
#   with open(sample_directory + row['UNIQUEID'] + '.txt', 'rb') as f:
#     wgs = f.readline().decode('ascii').strip()

#   rrdr_start, primer_used = get_start(wgs, primers)
#   primers_used[primer_used].append(row['UNIQUEID'])
#   if primer_used == -1: continue

#   unique_ids.append(row['UNIQUEID'])
#   seqs.append(wgs[rrdr_start - bp_up:rrdr_start + bp_down])

# data_dir = 'test/'

# np.save(evo_general_dir + data_dir + 'unique_ids.npy', unique_ids)
# np.save(evo_general_dir + data_dir + 'seqs.npy', seqs)

# data_dir = 'test/'
# unique_ids = np.load(evo_general_dir + data_dir + 'unique_ids.npy')
# seqs = np.load(evo_general_dir + data_dir + 'seqs.npy')

# embeds = []
# for i, seq in tqdm(enumerate(seqs)):
#   if i % 1000 == 0 and i != 0:
#     np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)
#     del embeds[:]

#   embeds.append(gef.run(seq)[0])

# # there are 12,287 samples, so the stragglers aren't saved yet
# np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)

In [ ]:
"""
  Save data in increments of 100.
"""

# data_dir = 'rif_embeddings_v1/embeds_1.0/'

# for i in tqdm(range(0, 12)): # because there are 12,287 samples
#   current = np.load(evo_general_dir + data_dir + 'embeds_' + str(i*1000 + 1000) + '.npy')
#   output_dir = evo_general_dir + 'rif_embeddings_v1/embeds_1.1_small/'
#   np.save(output_dir + 'embeds_' + str(i*1000) + '_to_' + str(i*1000 + 999) + '_index_0.npy', current[:,0,:])

#   output_dir = evo_general_dir + 'rif_embeddings_v1/embeds_1.0_chunks/'
#   for j in range(10):
#     file_name = 'embeds_' + str(i*1000 + j*100) + '_to_' + str(i*1000 + (j+1)*100 - 1) + '.npy'
#     np.save(output_dir + file_name, current[j*100:(j+1)*100,:,:])

# current = np.load(evo_general_dir + data_dir + 'embeds_12259.npy')
# output_dir = evo_general_dir + 'rif_embeddings_v1/embeds_1.1_small/embeds_12000_to_12258_index_0.npy'
# np.save(output_dir + file_name, current[:,0,:])

# output_dir = evo_general_dir + 'rif_embeddings_v1/embeds_1.0_chunks/embeds_12000_to_12258_index_0.npy'
# np.save(output_dir, current[12000:,:,:])
# # note: the last file must be handled separately

# Experiment 3: Compare sub-embedding

In our prior experiment, due to compute restraints, we used the 0th index of a sample's (500, 4096) shape embedding. (Note that there is 1 sub-embedding per bp in our input sequence.) However, we do not know if this is optimal or not. Before we attempt to test all 500 options (a computationally intensive and annoying task), we ask: is the *i*th sub-embedding all that different than the *j*th?

**Result:** Unsurprisingly, the answer is yes, each sub-embedding is different from the other. Cosine similarity for any given pair is 0.

In [ ]:
ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

NameError: name 'evo_general_dir' is not defined

In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
import numpy as np
from numpy.linalg import norm

data_dir = '/content/drive/MyDrive/EVO/rif_embeddings_v1/embeds_1.0_chunks/'

def get_file(start, end):
  """
    Helper function to make finding
  """
  return 'embeds_' + str(start) + '_to_' + str(end) + '.npy'

chunk = np.load(data_dir + get_file(0, 99))

def cosine_similarity(a, b):
  return np.dot(a, b) / (norm(a) * norm(b))

def get_length(sample):
  return np.array([norm(v) for v in sample])

def get_sim(sample):
  n = len(sample)
  result = np.array([[0 for _ in range(n)] for _ in range(n)])

  for i in range(n):
    for j in range(n):
      result[i][j] = cosine_similarity(sample[i], sample[j])

  return result

# Experiment 4: Predict MXF/EMB resistance with gyrA/embB


In [ ]:
# """
#   Creating a class so that it becomes easier to create
#   embeddings. Basically, you pass the model a primer, and
#   embeddings should pop out.

#   This does become a small challenge, because we need to
#   replace sequences as a part of our gene experiments.
# """
# def get_site(unique_id):
#   return unique_id.split('.')[1]

# def get_start(seq, primers):
#   for i, primer in enumerate(primers):
#     if primer in seq:
#       return seq.find(primer), i
#   return -1, -1

# class SequenceFinder():
#   def __init__(self,
#                genomes_dir,
#                bp_up=250,
#                bp_down=250,
#                vcf_df=None
#                )
#     self.genomes_dir = genomes_dir
#     self.bp_up = bp_up
#     self.bp_down = bp_down
#     self.vcf_df = vcf_df

#     self.logs = []

#   def save_sequences(self,
#                      primers,
#                      output_dir,
#                      logging=False):
#     assert vcf_df is not None, "No vcf_df has been defined."

#     unique_ids = []
#     seqs = []
#     genome_dir = 'genomes/'
#     for i, row in tqdm(vcf_df.iterrows()):
#       if row['UNIQUEID'] in unique_ids: continue

#     sample_directory = self.genomes_dir + get_site(row['UNIQUEID']) + '/'

#     with open(sample_directory + row['UNIQUEID'] + '.txt', 'rb') as f:
#     wgs = f.readline().decode('ascii').strip()

#     start, primer_used = get_start(wgs, primers)
#     primers_used[primer_used].append(row['UNIQUEID'])
#     if primer_used == -1:
#       count += 1
#       continue

In [ ]:
# ref_seq = 'atgacagacacgacgttgccgcctgacgactcgctcgaccggatcgaaccggttgacatcgagcaggagatgcagcgcagctacatcgactatgcgatgagcgtgatcgtcggccgcgcgctgccggaggtgcgcgacgggctcaagcccgtgcatcgccgggtgctctatgcaatgttcgattccggcttccgcccggaccgcagccacgccaagtcggcccggtcggttgccgagaccatgggcaactaccacccgcacggcgacgcgtcgatctacgacagcctggtgcgcatggcccagccctggtcgctgcgctacccgctggtggacggccagggcaacttcggctcgccaggcaatgacccaccggcggcgatgaggtacaccgaagcccggctgaccccgttggcgatggagatgctgagggaaatcgacgaggagacagtcgatttcatccctaactacgacggccgggtgcaagagccgacggtgctacccagccggttccccaacctgctggccaacgggtcaggcggcatcgcggtcggcatggcaaccaatatcccgccgcacaacctgcgtgagctggccgacgcggtgttctgggcgctggagaatcacgacgccgacgaagaggagaccctggccgcggtcatggggcgggttaaaggcccggacttcccgaccgccggactgatcgtcggatcccagggcaccgctgatgcctacaaaactggccgcggctccattcgaatgcgcggagttgttgaggtagaagaggattcccgcggtcgtacctcgctggtgatcaccgagttgccgtatcaggtcaaccacgacaacttcatcacttcgatcgccgaacaggtccgagacggcaagctggccggcatttccaacattgaggaccagtctagcgatcgggtcggtttacgcatcgtcatcgagatcaagcgcgatgcggtggccaaggtggtgatcaataacctttacaagcacacccagctgcagaccagctttggcgccaacatgctagcgatcgtcgacggggtgccgcgcacgctgcggctggaccagctgatccgctattacgttgaccaccaactcgacgtcattgtgcggcgcaccacctaccggctgcgcaaggcaaacgagcgagcccacattctgcgcggcctggttaaagcgctcgacgcgctggacgaggtcattgcactgatccgggcgtcggagaccgtcgatatcgcccgggccggactgatcgagctgctcgacatcgacgagatccaggcccaggcaatcctggacatgcagttgcggcgcctggccgcactggaacgccagcgcatcatcgacgacctggccaaaatcgaggccgagatcgccgatctggaagacatcctggcaaaacccgagcggcagcgtgggatcgtgcgcgacgaactcgccgaaatcgtggacaggcacggcgacgaccggcgtacccggatcatcgcggccgacggagacgtcagcgacgaggatttgatcgcccgcgaggacgtcgttgtcactatcaccgaaacgggatacgccaagcgcaccaagaccgatctgtatcgcagccagaaacgcggcggcaagggcgtgcagggtgcggggttgaagcaggacgacatcgtcgcgcacttcttcgtgtgctccacccacgatttgatcctgttcttcaccacccagggacgggtttatcgggccaaggcctacgacttgcccgaggcctcccggacggcgcgcgggcagcacgtggccaacctgttagccttccagcccgaggaacgcatcgcccaggtcatccagattcgcggctacaccgacgccccgtacctggtgctggccactcgcaacgggctggtgaaaaagtccaagctgaccgacttcgactccaatcgctcgggcggaatcgtggcggtcaacctgcgcgacaacgacgagctggtcggtgcggtgctgtgttcggccggcgacgacctgctgctggtctcggccaacgggcagtccatcaggttctcggcgaccgacgaggcgctgcggccaatgggtcgtgccacctcgggtgtgcagggcatgcggttcaatatcgacgaccggctgctgtcgctgaacgtcgtgcgtgaaggcacctatctgctggtggcgacgtcagggggctatgcgaaacgtaccgcgatcgaggaatacccggtacagggccgcggcggtaaaggtgtgctgacggtcatgtacgaccgccggcgcggcaggttggttggggcgttgattgtcgacgacgacagcgagctgtatgccgtcacttccggcggtggcgtgatccgcaccgcggcacgccaggttcgcaaggcgggacggcagaccaagggtgttcggttgatgaatctgggcgagggcgacacactgttggccatcgcgcgcaacgccgaagaaagtggcgacgataatgccgtggacgccaacggcgcagaccagacgggcaattaa'
# ref_seq = ref_seq.upper()
# print(len(ref_seq))

2517


In [ ]:
reuse_vcf = pd.read_csv(evo_general_dir + cryptic_general_file)
reuse_vcf = reuse_vcf[['UNIQUEID', 'EMB_BINARY_PHENOTYPE', 'EMB_PHENOTYPE_QUALITY', 'EMB_MIC']]

data_dir = 'emb_embeddings_v1/'

unique_ids = np.load(evo_general_dir + data_dir + 'unique_ids.npy')
seqs = np.load(evo_general_dir + data_dir + 'seqs.npy')

assert len(unique_ids) == len(seqs), "There are more unique_ids than seqs, or vice versa!"

In [ ]:
ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

configuration_hyena.py:   0%|          | 0.00/3.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/togethercomputer/evo-1-131k-base:
- configuration_hyena.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hyena.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
def get_site(unique_id):
  return unique_id.split('.')[1]

embedding_dir = evo_general_dir + 'emb_embeddings_v1/'
for i, id in tqdm(enumerate(unique_ids)):
  # out_sub_dir = 'embeds_1.0_left/'
  # out_dir = embedding_dir + out_sub_dir + 'site_' + get_site(id) + '/'
  # if not os.path.exists(out_dir):
  #   os.makedirs(out_dir)

  # if os.path.exists(out_dir + id + '.npy'): continue
  # embed = gef.run(seqs[i][:500])
  # np.save(out_dir + id + '.npy', embed)

  # out_sub_dir = 'embeds_1.0_right/'
  # out_dir = embedding_dir + out_sub_dir + 'site_' + get_site(id) + '/'
  # if not os.path.exists(out_dir):
  #   os.makedirs(out_dir)

  # if os.path.exists(out_dir + id + '.npy'): continue
  # embed = gef.run(seqs[i][-500:])
  # np.save(out_dir + id + '.npy', embed)

  out_sub_dir = 'embeds_1.0_full/'
  out_dir = embedding_dir + out_sub_dir + 'site_' + get_site(id) + '/'
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)
  embed = gef.run(seqs[i])
  np.save(out_dir + id + '.npy', embed)

12230it [3:17:37,  1.03it/s]


In [ ]:
embs_df = pd.DataFrame(unique_ids, columns=['UNIQUEID'])
embs_df = pd.merge(embs_df, reuse_vcf, on='UNIQUEID')
embs_df.to_csv(evo_general_dir + 'emb_embeddings_v1/' + 'embs_df.csv', index=False)

In [ ]:
def get_site(unique_id):
  return unique_id.split('.')[1]

def get_start(seq, primers):
  for i, primer in enumerate(primers):
    if primer in seq:
      return seq.find(primer), i
  return -1, -1

# we are interested in the 90th or so codon of the gyrA gene,
# i.e. the 270th or so nucleotide. thus, we want to start about 21 bases after the start
# of gyrA.
embB_start_seq = ref_seq[870:890]

In [ ]:
data_dir = 'emb_embeddings_v1/embeds_1.0/'

for i in tqdm(range(0, 12)): # because there are ~12,000 samples
  current = np.load(evo_general_dir + data_dir + 'embeds_' + str(i*1000 + 1000) + '.npy')
  output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.1_small/'
  np.save(output_dir + 'embeds_' + str(i*1000) + '_to_' + str(i*1000 + 999) + '_index_0.npy', current[:,0,:])

  output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.0_chunks/'
  for j in range(10):
    file_name = 'embeds_' + str(i*1000 + j*100) + '_to_' + str(i*1000 + (j+1)*100 - 1) + '.npy'
    np.save(output_dir + file_name, current[j*100:(j+1)*100,:,:])

# update this section mannually
current = np.load(evo_general_dir + data_dir + 'embeds_12261.npy')
output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.1_small/embeds_12000_to_12261_index_0.npy'
np.save(output_dir + file_name, current[:,0,:])

output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.0_chunks/embeds_12000_to_12261_index_0.npy'
np.save(output_dir + file_name, current[12000:,:,:])
# note: the last file must be handled separately

In [ ]:
def get_site(unique_id):
  return unique_id.split('.')[1]

def get_start(seq, primers):
  for i, primer in enumerate(primers):
    if primer in seq:
      return seq.find(primer), i
  return -1, -1

# we are interested in the 90th or so codon of the gyrA gene,
# i.e. the 270th or so nucleotide. thus, we want to start about 21 bases after the start
# of gyrA.
embB_start_seq = ref_seq[870:890]

"""
  Pretty much the same code as before, same deal,
  just extracting 500-bp sequences for embB.
"""
import numpy as np

primers = [embB_start_seq]
primers_used = [[] for _ in range(len(primers))]

# use the last 500 and the first 500 bp
# for individual embeddings
bp_up = 220
bp_down = 880

unique_ids = []
seqs = []
genome_dir = 'genomes/'
count = 0
for i, row in tqdm(reuse_vcf.iterrows()):
  if row['UNIQUEID'] in unique_ids: continue # i.e. we have already processed this file

  sample_directory = evo_general_dir + genome_dir + "site_" + get_site(row['UNIQUEID']) + '/'
  with open(sample_directory + row['UNIQUEID'] + '.txt', 'rb') as f:
    wgs = f.readline().decode('ascii').strip()

  start, primer_used = get_start(wgs, primers)
  primers_used[primer_used].append(row['UNIQUEID'])
  if primer_used == -1:
    count += 1
    continue

  unique_ids.append(row['UNIQUEID'])
  seqs.append(wgs[start - bp_up:start + bp_down])

data_dir = 'emb_embeddings_v1/'

np.save(evo_general_dir + data_dir + 'unique_ids.npy', unique_ids)
np.save(evo_general_dir + data_dir + 'seqs.npy', seqs)

12287it [3:48:34,  1.12s/it]


In [ ]:
np.save(evo_general_dir + data_dir + 'unique_ids.npy', unique_ids)
np.save(evo_general_dir + data_dir + 'seqs.npy', seqs)

In [ ]:
unique_ids = np.load(evo_general_dir + data_dir + 'unique_ids.npy')
seqs = np.load(evo_general_dir + data_dir + 'seqs.npy')

NameError: name 'data_dir' is not defined

# Scratchwork from a different notebook

In [ ]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9363010/table/pbio.3001721.t001/?report=objectonly

import pickle
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

evo_general_dir = '/content/drive/MyDrive/EVO/'
vcfs_dir = 'vcfs/'
cryptic_dir = 'cryptic_data/'
cryptic_reuse_csv = 'CRyPTIC_reuse_table_20231208.csv'

df = pd.read_csv(evo_general_dir + cryptic_reuse_csv)

In [ ]:
!pip install evo-model

import torch
from evo import Evo

!pip install gumpy

import gumpy as gp
from collections import Counter


class CustomEmbedding(torch.nn.Module):
  """
    Monkey patch to obtain Evo model embeddings,
    instead of logits.
  """
  def unembed(self, u):
    return u


class GeneEmbeddingFactory():
  """
    Just a wrapper around Evo. Because we call Evo often enough, this
    makes our lives marginally better.

    22.5 GB of GPU RAM is barely enough to perform inference on 500-length
    sequences. (Storing the model requires ~12, running inference requires ~10.)
  """
  def __init__(self,
               ref_genome, # Genome
               version='evo-1-8k-base',
               device='cuda:0',
               logits=False):

    self.ref_genome = ref_genome

    evo_model = Evo(version)
    self.model = evo_model.model
    self.tokenizer = evo_model.tokenizer

    self.device = device
    self.model.to(device)
    self.model.eval()

    if not logits:
      self.model.unembed = CustomEmbedding()


  def get_gene(self, vcf, gene_name):
    """
      Note: Unless you have a lot of RAM, it is not possible
      load 12,228 whole M. tb genomes. Hence, we work on a gene-by-gene
      basis.

      input:
      - vcf: gumpy VCFFile object, M. tb VCF
      - gene_name: string

      output:
      - gene: gumpy Gene object
    """
    genome = self.ref_genome + vcf

    return genome.build_gene(gene_name)

  def run(self,
          sequence,
          to_cpu=True):
    input_ids = torch.tensor(
        self.tokenizer.tokenize(sequence),
        dtype=torch.int,
    ).to(self.device).unsqueeze(0)

    embed, _ = self.model(input_ids) # (batch, length, embed dim)
    if to_cpu: embed = embed.to(dtype=torch.float64).detach().cpu().numpy()

    return embed

In [ ]:
ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

reuse_vcf = pd.read_csv(evo_general_dir + cryptic_reuse_csv)
reuse_vcf = reuse_vcf[['UNIQUEID', 'VCF', 'MXF_BINARY_PHENOTYPE', 'MXF_PHENOTYPE_QUALITY', 'MXF_MIC']]

# # Did you know? There is an 81-bp section of the M. tb genome
# # known as the riframin resistance determining region (RRDR).
# # We will be embedding the RRDR for our first experiment.
# tb_rrdr_primer1 = 'CGATCACACCGCAGACGTTGA'
# tb_rrdr_primer2 = 'CAGACGTTGATCAACATCCG'

# # the RRDR is in the rpoB gene
# gene_name = 'rpoB'
# bp_up = 250
# bp_down = 250

In [ ]:
# def get_site(unique_id):
#   return unique_id.split('.')[1]

# def get_start(seq, primers):
#   for i, primer in enumerate(primers):
#     if primer in seq:
#       return seq.find(primer), i
#   return -1, -1

# primers = [tb_rrdr_primer1, tb_rrdr_primer2]
# primers_used = [[] for _ in range(len(primers))]

# unique_ids = []
# seqs = []
# genome_dir = 'genomes/'
# for i, row in tqdm(reuse_vcf.iterrows()):
#   if row['UNIQUEID'] in unique_ids: continue

#   sample_directory = evo_general_dir + genome_dir + "site_" + get_site(row['UNIQUEID']) + '/'
#   with open(sample_directory + row['UNIQUEID'] + '.txt', 'rb') as f:
#     wgs = f.readline().decode('ascii').strip()

#   rrdr_start, primer_used = get_start(wgs, primers)
#   primers_used[primer_used].append(row['UNIQUEID'])
#   if primer_used == -1: continue

#   unique_ids.append(row['UNIQUEID'])
#   seqs.append(wgs[rrdr_start - bp_up:rrdr_start + bp_down])

In [ ]:
import numpy as np

In [ ]:
ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

configuration_hyena.py:   0%|          | 0.00/3.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/togethercomputer/evo-1-131k-base:
- configuration_hyena.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hyena.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
# data_dir = 'test/'

# np.save(evo_general_dir + data_dir + 'unique_ids.npy', unique_ids)
# np.save(evo_general_dir + data_dir + 'seqs.npy', seqs)

data_dir = 'mxn_embeddings_v1/'
unique_ids = np.load(evo_general_dir + data_dir + 'unique_ids.npy')
seqs = np.load(evo_general_dir + data_dir + 'seqs.npy')

In [ ]:
missing_seqs = ['site.02.subj.1120.lab.2014185039.iso.1',
                'site.03.subj.T164.lab.T164.iso.1',
                'site.04.subj.00873.lab.714114.iso.1',
                'site.05.subj.CA-1143.lab.CO-02470-19.iso.1',
                'site.05.subj.LS-1089.lab.LS-10870-18.iso.1',
                'site.05.subj.PTAN-0174.lab.TAN-521.iso.1',
                'site.06.subj.06TB_1076.lab.06MIL2196.iso.1',
                'site.06.subj.SKT_0001-14.lab.06MIL0414.iso.1',
                'site.08.subj.05TB33004.lab.1770.iso.1',
                'site.10.subj.YA00024316.lab.YA00024316.iso.1',
                'site.10.subj.YA00162430.lab.YA00162430.iso.1',
                'site.20.subj.SA00694508.lab.YA00135489.iso.1']

In [ ]:
indices = np.where(np.isin(unique_ids, missing_seqs))[0]

In [ ]:
def get_site(unique_id):
  return unique_id.split('.')[1]

In [ ]:
evo_general_dir

'/content/drive/MyDrive/EVO/'

In [ ]:
print(evo_general_dir + "mxn_embeddings_v1/embeds_1.0_singles/site_" + get_site(missing_seqs[i]) + "/" + missing_seqs[i] + ".npy")

/content/drive/MyDrive/EVO/mxn_embeddings_v1/embeds_1.0_singles/site_20/site.20.subj.SA00694508.lab.YA00135489.iso.1.npy


In [ ]:
for i in tqdm(range(len(missing_seqs))):
  embed = gef.run(seqs[i])
  np.save(evo_general_dir + "mxn_embeddings_v1/embeds_1.0_singles/site_"
          + get_site(missing_seqs[i]) + "/" + missing_seqs[i] + ".npy", embed)
  # np.save(evo_general_dir + missing_seqs[i] + '.npy', embed)
  # break

100%|██████████| 12/12 [00:11<00:00,  1.02it/s]


In [ ]:
print(evo_general_dir + "mxn_embeddings_v1/embeds_1.0_singles/site_"
          + get_site(missing_seqs[i]) + "/" + missing_seqs[i] + ".npy")

/content/drive/MyDrive/EVO/mxn_embeddings_v1/embeds_1.0_singles/site_20/site.20.subj.SA00694508.lab.YA00135489.iso.1.npy


In [ ]:
embeds = []
for i, seq in tqdm(enumerate(seqs)):
  if i % 1000 == 0 and i != 0:
    np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)
    del embeds[:]

  embeds.append(gef.run(seq)[0])

# there are 12,262 samples, so the stragglers aren't saved yet
np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)

In [ ]:
data_dir = 'mxn_embeddings_v1/embeds_1.0/'

for unique_id in tqdm(unique_ids):

for i in tqdm(range(0, 12)): # because there are ~12,000 samples
  current = np.load(evo_general_dir + data_dir + 'embeds_' + str(i*1000 + 1000) + '.npy')

  output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.0_chunks/'
  for j in range(10):
    file_name = 'embeds_' + str(i*1000 + j*100) + '_to_' + str(i*1000 + (j+1)*100 - 1) + '.npy'
    np.save(output_dir + file_name, current[j*100:(j+1)*100,:,:])

# update this section mannually
# current = np.load(evo_general_dir + data_dir + 'embeds_12261.npy')
# output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.1_small/embeds_12000_to_12261_index_0.npy'
# np.save(output_dir + file_name, current[:,0,:])

# output_dir = evo_general_dir + 'mxn_embeddings_v1/embeds_1.0_chunks/embeds_12000_to_12261_index_0.npy'
# np.save(output_dir + file_name, current[12000:,:,:])
# note: the last file must be handled separately

In [ ]:
import pandas as pd
out = pd.DataFrame(unique_ids, columns=['UNIQUEID'])
out = pd.merge(out, reuse_vcf, how='left', on='UNIQUEID')

In [ ]:
# embeds = []
# for i, seq in tqdm(enumerate(seqs)):
#   if i % 1000 == 0 and i != 0:
#     np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)
#     del embeds[:]

#   embeds.append(gef.run(seq)[0])

# # there are 12,287 samples, so the stragglers aren't saved yet
# np.save(evo_general_dir + data_dir + 'embeds_' + str(i) + '.npy', embeds)

# Deprecated - DO NOT USE

In [ ]:
# """
#   Note: This code is actually way too slow. It was used to organize segments
#   of rpoB that we are interested in. However, using Gumpy was way too slow,
#   because (unsurprisingly) working with whole genomes is slow. Instead,
#   we moved towards saving a sample's whole genome, then just searching for the relevant primers.
# """
# def get_site(unique_id):
#   return unique_id.split('.')[1]

# output = []
# for i, row in tqdm(reuse_vcf.iterrows()):
#   if i > 10: break
#   sample_directory = evo_general_dir + vcfs_dir + "site_" + get_site(row['UNIQUEID']) + '/'

#   assert os.path.exists(sample_directory), "Directory does not exist for sample " + row['UNIQUEID']

#   file_name = row['VCF'].strip().split('/')[-1]
#   assert os.path.exists(sample_directory + file_name), "File does not exist for sample " + row['UNIQUEID']

#   file_path = sample_directory + file_name
#   gunzip(file_path)

#   vcf = gp.VCFFile(file_path[:-3])
#   gzip(file_path[:-3])

#   gene = gef.get_gene(vcf, gene_name)
#   gene_seq = "".join(gene.nucleotide_sequence).upper()

#   rrdr_start = ""
#   if tb_rrdr_primer1 in gene_seq:
#     rrdr_start = gene_seq.find(tb_rrdr_primer1)
#   elif tb_rrdr_primer2 in gene_seq:
#     rrdr_start = gene_seq.find(tb_rrdr_primer2)
#   else:
#     print('Error at row ' + str(i) + '. ' + row['UNIQUEID'] + ' does not have a RRDR primer.')
#     output.append([row['UNIQUEID'], "MISSING RRDR PRIMER"])
#     continue

#   # if i % 1000 == 0:
#   #   o = np.asarray(output)
#   #   np.savetxt(evo_general_dir + 'test/' + 'checkpoint_' + str(i) + '.csv', o, delimiter=',', fmt='%s')

#   output.append([row['UNIQUEID'], gene_seq[rrdr_start - bp_up:rrdr_start + bp_down]])

In [ ]:
# """
#   This block of code performs the following steps:
#   1. Open/closes a VCF
#   2. "Adds" the VCF to the reference genome (thus obtaining the sample's WGS)
#   3. Uses a primer to find the RRDR
#   4. Runs Evo on the RRDR

#   However, this code shouldn't be used anymore. In particular, it requires too much
#   time to "add" a VCF to the refence genome. Instead, whole genomes have been
#   built and saved to a separate directory named "genomes".

#   ... For reference, generating "genomes" took 24 hours. This is due, in part,
#   because gumpy is on the slower side. But not much can be done there.
# """

def get_site(unique_id):
  return unique_id.split('.')[1]

output = []
for i, row in tqdm(reuse_vcf.iterrows()):
  sample_directory = evo_general_dir + vcfs_dir + "site_" + get_site(row['UNIQUEID']) + '/'

  assert os.path.exists(sample_directory), "Directory does not exist for sample " + row['UNIQUEID']

  file_name = row['VCF'].strip().split('/')[-1]
  assert os.path.exists(sample_directory + file_name), "File does not exist for sample " + row['UNIQUEID']

  file_path = sample_directory + file_name
  gunzip(file_path)

  vcf = gp.VCFFile(file_path[:-3])
  gzip(file_path[:-3])

  gene = gef.get_gene(vcf, gene_name)
  gene_seq = "".join(gene.nucleotide_sequence).upper()

  rrdr_start = ""
  if tb_rrdr_primer1 in gene_seq:
    rrdr_start = gene_seq.find(tb_rrdr_primer1)
  elif tb_rrdr_primer2 in gene_seq:
    rrdr_start = gene_seq.find(tb_rrdr_primer2)
  else:
    print('Error at row ' + str(i) + '. ' + row['UNIQUEID'] + ' does not have a RRDR primer.')
    output.append([row['UNIQUEID'], "MISSING RRDR PRIMER"])
    continue

  if i % 1000 == 0:
    o = np.asarray(output)
    np.savetxt(evo_general_dir + 'test/' + 'checkpoint_' + str(i) + '.csv', o, delimiter=',', fmt='%s')

  output.append([row['UNIQUEID'], gene_seq[rrdr_start - bp_up:rrdr_start + bp_down]])
  # embed = gef.run(gene_seq[rrdr_start - bp_up:rrdr_start + bp_down])
  # output.append([row['UNIQUEID'], embed])

Mini-summary of results, thus far.
* There are a total of 128 variants
* Of these, only 6 are represented in more than 20 samples
* They can occur pretty much anywhere in the rpoB genome (i.e. variants are present at the start and the end of the sequence).



In [ ]:
"""
  Code to create an instance of the Evo model
"""
# Uncomment if you need to install the model!
!pip install evo-model

from evo import Evo
import torch

device = 'cuda:0'

# evo_model = Evo('evo-1-131k-base') # load model
evo_model = Evo('evo-1-8k-base') # load model
model, tokenizer = evo_model.model, evo_model.tokenizer
model.to(device)
model.eval()

# Note to self: How does this code work?
# How to extract embeddings: https://github.com/evo-design/evo/issues/32
from torch import nn

class CustomEmbedding(nn.Module):
  def unembed(self, u):
    return u

model.unembed = CustomEmbedding()

# Basically, this removes the last part of Evo
# so that the embeddings are not turned into logits.

# # Sample code from the Evo GitHub
# sequence = 'ACGT'
# input_ids = torch.tensor(
#     tokenizer.tokenize(sequence),
#     dtype=torch.int,
# ).to(device).unsqueeze(0)

# embed, _ = model(input_ids) # (batch, length, embed dim)

# print('Embed: ', embed)
# print('Shape (batch, length, embed dim): ', embed.shape)

In [ ]:
"""
  This section was used to determine Evo GPU use. In short,
  1000 bp inputs require about 10GB of GPU RAM. 1500 bp requires about
  20 GB. So, we recommend running Evo on sequences of at most 500
  (if we use Colab).
"""

# check how much GPU we have
# A100s tend to be alloted 40 GB of GPU RAM
# L4s tend to be alloted around 24 GB
!pip install gputil

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

In [ ]:
"""
  This code was replaced by the use of gumpy.
"""
# def exp_2_open_vcf(file_path, stop):
#   """
#     open_vcf uses pd.read_csv, which is too slow for our purposes.
#     This script stops prematurely.
#   """
#   with open(file_path) as f:
#     POS_COL_ID = 1
#     cols = []
#     while '#' in f.readline():
#       cols = f.readline().strip().split('\t')
#       cols[0] = cols[0][1:]

#     result = list()
#     for line in f:
#       line = line.strip().split('\t')
#       if int(line[POS_COL_ID]) > stop: break
#       result.append(line)

#     return pd.DataFrame(result, columns=cols)

# # opening VCFs takes a LONG time (1 hour). (This is because
# # it takes a long time for pandas to load a VCF. We could improve
# # this, especially when we do not need the ENTIRE VCF.)
# df_sample = df.sample(n=100, random_state=42)

# # first, let us check what variants we observe in our data
# vcfs = []
# for index, row in tqdm(df_sample.iterrows()):
#   file_path = get_sample_file_path(row)
#   gunzip(file_path)
#   vcfs.append(exp_2_open_vcf(file_path[:-3], rpob_end))
#   gzip(file_path[:-3])

# # filter each VCF so we only have the variants we care about
# filtered_vcfs = []
# for i, vcf in enumerate(vcfs):
#   vcf['POS'] = pd.to_numeric(vcf['POS'])
#   filtered_vcf = vcf.query('POS >= @rpob_start and POS <= @rpob_end')
#   filtered_vcfs.append(filtered_vcf)

# from collections import Counter

# # check what variants we have
# variants = set()
# counter = Counter()
# for i, vcf in enumerate(filtered_vcfs):
#   for index, row in vcf.iterrows():
#     counter[row['POS']] += 1
#     variants.add(row['POS'])

# # from pprint import pprint

# min_threshold = 20
# filtered_variants = {x: count for x, count in counter.items() if count >= min_threshold}
# print("Number of variants with more than " + str(min_threshold) + " samples:", len(filtered_variants))

# print("Range of ALL variant positions:", max(counter.keys()) -  min(counter.keys()))
# print("Range of filtered variant positions:", max(filtered_variants.keys()) -  min(filtered_variants.keys()))

In [ ]:
"""
  This section (in particular, GenomeLoader) was written to
  turn VCFs into whole-genome sequences. There are likely tools to do
  this already (e.g. hail, a package managed by Harvard folks, is
  designed to manage VCFs), but were hard for me to figure out.

  Instead of using this class, use the gumpy package: https://pypi.org/project/gumpy/
  (I believe this is what the CRyPTIC folks used.)
"""

def get_fragment(df_vcf, index, genome_kvs, bp_down, bp_up):
  """
    For testing....

    In the future, this can be repurposed to nab specific areas
    of a genome.

    inputs:
    - df_vcf: DataFrame representing the body of a VCF file
    - index: row of VCF file
    - genome_kvs: dict that stores genome fragments created by GenomeLoader
    - bp_down, bp_up: positive integers (i.e. > 0)
  """
  key_after = df_vcf.loc[index]['POS'] + len(df_vcf.loc[index]['REF']) - 1
  key_before = df_vcf.loc[index - 1]['POS'] + len(df_vcf.loc[index - 1]['REF']) - 1

  frag_before = genome_kvs[key_before]
  frag_after = genome_kvs[key_after]

  return frag_before[-bp_down:] + frag_after[:bp_up]

# Example tests, need to build out this section
# pos = df_vcf.loc[525]['POS']
# print(gl.reference_genome[pos+7:pos+28])

# print(get_fragment(df_vcf, 525, gl.frag_genomes[cols[-1]], 6, 20)[:])

class GenomeLoader():
  def __init__(self, whole_genome_path=None, reference_genome=None):
    self.whole_genome_path = whole_genome_path
    self.reference_genome = reference_genome
    self.frag_genomes = dict()

    if self.whole_genome_path:
      with open(whole_genome_path) as f:
        metadata = f.readline().strip()
        lines = [line.strip() for line in f]
        self.reference_genome = "".join(lines)

    assert self.reference_genome, "GenomeLoader did not receive a reference genome"


  def vcf_to_genome(self, vcf_file):
    """
      Given a vcf_file, return a whole genome.

      Here is the general strategy:
      Think of the reference genome as a series of fragments;
      we create fragments such that each fragment ends in a mutation
      according to the vcf_file.

      Then, to re-construct a whole genome, we append these fragments
      together. Done!
    """
    f = open(vcf_file)

    # VCF files have a detailed header; each row of the header begins with ##
    # The final row is the column header; it begins with #
    cols = []
    while '#' in f.readline():
      cols = f.readline().strip().split('\t')
    cols[0] = cols[0][1:]

    df_vcf = pd.read_csv(f, delimiter='\t', header=0, names=cols)

    genome_kvs = dict()
    current_pos = 0
    for index, row in df_vcf.iterrows():
      current_fragment = self.reference_genome[current_pos:row['POS'] - 1]
      current_fragment += row['ALT']

      genome_kvs[current_pos] = current_fragment
      current_pos = row['POS'] + len(row['REF']) - 1

    genome = ""
    for key, value in genome_kvs.items(): #requires Python 3.7
      genome += value

    self.frag_genomes[cols[-1]] = genome_kvs

    return genome